# INFO 4271 - Exercise 7 - Evaluation

Issued: June 4, 2024

Due: June 10, 2024

Please submit this filled sheet via Ilias by the due date.

---

# 1. Metric Calibration
Search evaluation scores are often bounded in the `[0,1]` interval to make them comparable. This range (wrongly!) suggests that these scores can be interpreted as probabilities. 

a) Implement the computation of average precision and compute AP scores for three example systems.

In [4]:
#The following three rankings have been produced by three independent search systems. 
# Each entry refers to the rank of the result, the document ID, the retrieval model score and 
# whether or not the document was judged relevant to this query.

S1 = [
    [1,  'D24', 0.99, 1],
    [2,  'D86', 0.98, 1],
    [3,  'D74', 0.95, 1],
    [4,  'D95', 0.95, 1],
    [5,  'D11', 0.93, 1],
    [6,  'D49', 0.89, 0],
    [7,  'D67', 0.89, 0],
    [8,  'D54', 0.87, 0],
    [9,  'D37', 0.83, 0],
    [10, 'D30', 0.81, 0]
]

S2 = [
    [1,  'D83', 0.99, 0],
    [2,  'D51', 0.81, 1],
    [3,  'D44', 0.80, 0],
    [4,  'D65', 0.75, 1],
    [5,  'D32', 0.71, 0],
    [6,  'D16', 0.68, 1],
    [7,  'D87', 0.63, 0],
    [8,  'D12', 0.21, 1],
    [9,  'D59', 0.20, 0],
    [10, 'D24', 0.18, 1]
]

S3 = [
    [1,  'D14', 0.89, 1],
    [2,  'D44', 0.81, 0],
    [3,  'D35', 0.81, 1],
    [4,  'D41', 0.72, 0],
    [5,  'D51', 0.71, 1],
    [6,  'D55', 0.67, 0],
    [7,  'D63', 0.63, 1],
    [8,  'D70', 0.60, 0],
    [9,  'D11', 0.57, 1],
    [10, 'D90', 0.55, 0]
]


# Compute Average Precision based on a ranked result list and 
# the overall number of relevant documents

def ap(ranking, R):
    num_relevant = 0
    total_precision = 0.0
    
    for i, result in enumerate(ranking, start=1):
        if result[3] == 1:
            num_relevant += 1
            precision = num_relevant / i
            total_precision += precision
    
    if num_relevant == 0:
        return 0.0
    
    return total_precision / R    


for system in [S1, S2, S3]:
    print(ap(system, 5))

1.0
0.5
0.6787301587301586


b) Now imagine a fourth system with the following score distribution. How should it be ranked relative to the original three systems?

```
S4 = [
    [1,  'D24', 0.90, 1],
    [2,  'D86', 0.80, 1],
    [3,  'D74', 0.85, 1],
    [4,  'D95', 0.75, 1],
    [5,  'D11', 0.68, 1],
    [6,  'D49', 0.62, 0],
    [7,  'D67', 0.55, 0],
    [8,  'D54', 0.40, 0],
    [9,  'D37', 0.34, 0],
    [10, 'D30', 0.27, 0]
]
```

Answer:

It should outperform both S2 and S3 since it places all 5 relevant documents at the top 5 places in the ranking. The AP would be the same compared to S1.

Considering the model retrieval score, we could point out 2 trends when comparing to S1:
- The scores of the relevant documents are lower compared to S1. The model of S1 is more sure about the actually relevant docs.
- The scores of the irrelevant documents are also lower compared to S1. That means that the model of S1 is more sure about retrieving documents that are actually not relevant. The model of S4 is more uncertain about lower ranked documents which would indicate better knowledge of whats less important. 

# 2. Query Performance Prediction

Imagine you had many pairs of queries Q and nDCG scores s for a retrieval system. Would you be able to skip the retrieval model and performance metric calculation and directly predict how well the system would do for a given query? Why does this (not) work? Discuss all pertinent considerations.

Answer:

Predicting the performance of a retrieval system for a given query only based on queries and nDCG scores is not feasible. While one could design this problem as a regression task that predicts nDCG scores given a query, we should consider:

- Lack of context: nDCG scores are calculated based on the relevance of the retrieved documents to the query. Without the retrieval model, the context in which the documents were retrieved is lost. This makes it difficult to accurately predict the nDCG score.

- Time: The relevance of documents can change over time. A document that is highly relevant to a query today may not be relevant in the future. Without the retrieval model, it would be difficult to account for these temporal dynamics.
